In [2]:
from numpy import zeros, ones, expand_dims, asarray
from numpy.random import randn, randint
from keras.models import Sequential, load_model,Model
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import LeakyReLU, Dropout
from keras.layers import BatchNormalization, Activation
from sklearn.model_selection import train_test_split
from keras import initializers
from keras.initializers import RandomNormal
from matplotlib import pyplot
import numpy as np
from math import sqrt
import pickle

In [3]:
#chargement des données
with open('liste_prot', 'rb') as f:
    X_train = pickle.load(f)
X_train = np.array(X_train)

In [4]:
#test,train
X_train,X_test = train_test_split(X_train,test_size = 0.2)

In [5]:
#700 proteines de 939 aa et 21 aa différents
X_test.shape

(700, 939, 21)

In [6]:
def build_autoencoder(in_shape=(939, 21, 1), size_mini = 10) :
    autoencoder = Sequential()
    #encoder
    autoencoder.add(Input(shape=in_shape))
    autoencoder.add(Flatten())
    autoencoder.add(Dense(512))
    autoencoder.add(LeakyReLU(alpha=0.2))
    autoencoder.add(Dropout(0.3))
    autoencoder.add(Dense(256))
    autoencoder.add(LeakyReLU(alpha=0.2))
    autoencoder.add(Dropout(0.3))
    autoencoder.add(Dense(size_mini))
    #decoder
    autoencoder.add(BatchNormalization())
    autoencoder.add(Dense(256))
    autoencoder.add(LeakyReLU(alpha=0.2))
    autoencoder.add(Dense(400))
    autoencoder.add(LeakyReLU(alpha=0.2))
    autoencoder.add(Dense(512))
    autoencoder.add(LeakyReLU(alpha=0.2))
    autoencoder.add(Dense(939 * 21 * 1,activation='sigmoid'))
    autoencoder.add(Reshape((939, 21, 1)))
    autoencoder.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
    return autoencoder

In [7]:
autoencoder = build_autoencoder()

In [8]:
autoencoder.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 19719)             0         
                                                                 
 dense (Dense)               (None, 512)               10096640  
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 512)               0         
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 256)               0         
                                                                 
 dropout_1 (Dropout)         (None, 256)               0

In [9]:
#fitting même entrée que la sortie
autoencoder.fit(X_train,X_train,epochs=100, batch_size=128,validation_data = (X_test,X_test))

Epoch 1/100
22/22 [==============================] - 4s 170ms/step - loss: 1.8566 - accuracy: 0.8979 - val_loss: 1.4949 - val_accuracy: 0.9549
Epoch 2/100
22/22 [==============================] - 4s 163ms/step - loss: 0.8263 - accuracy: 0.9516 - val_loss: 1.0665 - val_accuracy: 0.9564
Epoch 3/100
22/22 [==============================] - 3s 157ms/step - loss: 0.7560 - accuracy: 0.9584 - val_loss: 0.8857 - val_accuracy: 0.9591
Epoch 4/100
22/22 [==============================] - 3s 155ms/step - loss: 0.7234 - accuracy: 0.9598 - val_loss: 0.7580 - val_accuracy: 0.9610
Epoch 5/100
22/22 [==============================] - 3s 152ms/step - loss: 0.6997 - accuracy: 0.9612 - val_loss: 0.7286 - val_accuracy: 0.9601
Epoch 6/100
22/22 [==============================] - 3s 156ms/step - loss: 0.6777 - accuracy: 0.9616 - val_loss: 0.6812 - val_accuracy: 0.9612
Epoch 7/100
22/22 [==============================] - 3s 156ms/step - loss: 0.6594 - accuracy: 0.9626 - val_loss: 0.6638 - val_accuracy: 0.9625

Epoch 58/100
22/22 [==============================] - 3s 152ms/step - loss: 0.3047 - accuracy: 0.9842 - val_loss: 0.3805 - val_accuracy: 0.9831
Epoch 59/100
22/22 [==============================] - 3s 154ms/step - loss: 0.3034 - accuracy: 0.9841 - val_loss: 0.3837 - val_accuracy: 0.9837
Epoch 60/100
22/22 [==============================] - 4s 162ms/step - loss: 0.2994 - accuracy: 0.9843 - val_loss: 0.3768 - val_accuracy: 0.9838
Epoch 61/100
22/22 [==============================] - 3s 157ms/step - loss: 0.2975 - accuracy: 0.9845 - val_loss: 0.3770 - val_accuracy: 0.9839
Epoch 62/100
22/22 [==============================] - 4s 163ms/step - loss: 0.2951 - accuracy: 0.9845 - val_loss: 0.3772 - val_accuracy: 0.9842
Epoch 63/100
22/22 [==============================] - 4s 162ms/step - loss: 0.2928 - accuracy: 0.9846 - val_loss: 0.3749 - val_accuracy: 0.9842
Epoch 64/100
22/22 [==============================] - 3s 152ms/step - loss: 0.2897 - accuracy: 0.9847 - val_loss: 0.3721 - val_accuracy:

In [10]:
#on récupère le decoder qui correspond aux dernières couches
decoder = Sequential()
decoder.add(Input(shape=10))
for layer in autoencoder.layers[8:]: decoder.add(layer)

In [11]:
decoder.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 10)               40        
 ormalization)                                                   
                                                                 
 dense_3 (Dense)             (None, 256)               2816      
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 256)               0         
                                                                 
 dense_4 (Dense)             (None, 400)               102800    
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 400)               0         
                                                                 
 dense_5 (Dense)             (None, 512)               205312    
                                                      

In [12]:
#on créée une nouvelle protéine
test = [[0,1,0,1,0,1,0,1,0,1]]
prot = decoder.predict(test)

In [13]:
prot.shape

(1, 939, 21, 1)

In [22]:
prot[:,85:86]

array([[[[0.66826266],
         [0.02801213],
         [0.0424239 ],
         [0.11522511],
         [0.00167561],
         [0.05848324],
         [0.17943785],
         [0.23706606],
         [0.0317345 ],
         [0.01482689],
         [0.17202678],
         [0.03613278],
         [0.03632024],
         [0.01300931],
         [0.06885821],
         [0.11332825],
         [0.07710719],
         [0.00740239],
         [0.01962546],
         [0.00907153],
         [0.09047353]]]], dtype=float32)

In [14]:
#to_do :
#fonction prenant des nombres aléatoires pour la création de nouvelles prot
#passer de AE à VAE en rajoutant l'étape de sampling
#fonction permettant de passer du tableau en 939*21 à une séquence type "HWY..."